In [1]:
midi_source_path = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI"
midi_lib_path = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/midi_lib"

In [2]:
import sys
sys.path.append(midi_lib_path)

import pandas as pd
import json
import os

import converter.hooktheory_utils as htu
from converter.hooktheory_json_to_midi_dto import hooktheory_json_song_part_to_midi_dto_converter
from converter.midi_dto_to_midi_file import midi_dto_to_midi_file_converter
from converter.chord_dto_to_note_dtos import chord_dto_to_note_dtos_converter
from converter.hooktheory_json_chord_to_chord_dto import hooktheory_json_chord_to_chord_dto_converter

from dto.KeySignature import KeySignatureDTO

import const.midi as mc

import midi_utils as mu

In [3]:
hooktheory_json_file_path_1 = f"{midi_source_path}/Ideas/hooktheory/data/spectrum.json"
hooktheory_json_file_path_2 = f"{midi_source_path}/Ideas/hooktheory/data/carolina.json"
# hooktheory_json_file_path_3 = f"{midi_source_path}/Ideas/hooktheory/data/one-bite.json"
hooktheory_json_file_path_3 = f"{midi_source_path}/Ideas/hooktheory/data/for-you.json"

hooktheory_json_1 = None
hooktheory_json_2 = None
hooktheory_json_3 = None

for hooktheory_json_file_path in [
    hooktheory_json_file_path_1, 
    hooktheory_json_file_path_2, 
    hooktheory_json_file_path_3
]:
    with open(hooktheory_json_file_path, "r") as f:
        hooktheory_json = json.load(f)

    if hooktheory_json_1 is None:
        hooktheory_json_1 = hooktheory_json
    elif hooktheory_json_2 is None:
        hooktheory_json_2 = hooktheory_json
    elif hooktheory_json_3 is None:
        hooktheory_json_3 = hooktheory_json

    print(hooktheory_json.keys())
    for song_data in hooktheory_json["song_parts"]:
        print(song_data.keys())
        print(song_data["metadata"].keys())
        print(song_data["main_data"].keys())

    print(json.dumps(hooktheory_json["song_parts"][0]["main_data"]["chords"]))
    print(json.dumps(hooktheory_json["song_parts"][0]["main_data"]["notes"]))
    print(json.dumps(hooktheory_json["song_parts"][0]["main_data"]["inactiveNotes"]))
    # print(json.dumps(hooktheory_json["song_parts"][0]["main_data"]["chords"], indent=4))
    # print(json.dumps(hooktheory_json["song_parts"][0]["main_data"]["notes"], indent=4))
    # print(json.dumps(hooktheory_json["song_parts"][0]["main_data"]["inactiveNotes"], indent=4))

dict_keys(['artist_name', 'artist_link', 'song_name', 'link', 'metrics', 'metrics_conclusion', 'key_conclusion', 'song_parts'])
dict_keys(['song_part_id', 'metadata', 'main_data_type', 'main_data'])
dict_keys(['ID', 'user_id', 'dateModified', 'HTID', 'artist', 'song', 'section', 'revision', 'chords', 'youTubeID', 'dotDNAbin', 'xmlType', 'SIF', 'SInE', 'SInD', 'SInED', 'SInA', 'beatsInMeasure', 'songKey', 'isPrivate', 'views', 'hasMelody', 'hasChords', 'isAnalysis', 'voteup', 'votedown', 'score', 'songURL', 'artistURL', 'sectionURL', 'editComments', 'visible', 'wikiorder', 'bIsPaired', 'YouTubeVideoID', 'bpm', 'mode', 'borrowedFixed', 'SInEDFixed', 'user_id_editor'])
dict_keys(['version', 'chords', 'notes', 'keys', 'tempos', 'meters', 'breaks', 'bands', 'lyrics', 'sections', 'cursor', 'settings', 'loopGui', 'youtube', 'keyFrames', 'activeMelodyIndex', 'endBeat', 'inactiveNotes', 'visibleMelodies', 'soloAndMutedMelodies', 'pickup', 'fp'])
[{"root": 6, "beat": 1, "duration": 2, "type": 5,

In [4]:
song_part_chord_df_1 = pd.DataFrame(hooktheory_json_1["song_parts"][0]["main_data"]["chords"])
song_part_chord_df_1.head()

,root,beat,duration,type,inversion,applied,adds,omits,alterations,suspensions,pedal,alternate,borrowed,isRest,recordingEndBeat
0,6,1.0,2.0,5,0,0,[9],[],[],[],None,,None,False,None
1,6,3.0,2.0,7,0,0,[],[],[],[],None,,None,False,None
2,6,5.0,2.0,5,0,0,[9],[],[],[],None,,None,False,None
3,1,7.0,2.0,7,3,0,[],[],[],[],None,,None,False,None
4,1,9.0,6.0,7,0,0,[],[],[],[],None,,None,False,None


In [5]:
song_part_chord_df_2 = pd.DataFrame(hooktheory_json_2["song_parts"][0]["main_data"]["chords"])
song_part_chord_df_2.head()

,root,beat,duration,type,inversion,applied,adds,omits,alterations,suspensions,pedal,alternate,borrowed,isRest,recordingEndBeat
0,1,6,1,5,0,0,[],[],[],[],None,,,False,None
1,1,8,1,5,0,0,[],[],[],[],None,,,False,None
2,1,10,1,5,0,0,[],[],[],[],None,,,False,None
3,1,12,1,5,0,0,[],[],[],[],None,,,False,None
4,7,14,1,5,0,0,[],[],[],[],None,,,False,None


In [6]:
for song_part_main_data in [
    hooktheory_json_1["song_parts"][0]["main_data"],
    hooktheory_json_2["song_parts"][0]["main_data"]
]:
    for k in ["chords", "notes", "inactiveNotes"]:
        print()
        print(f"-----> {k.upper()}:")
        if k == "inactiveNotes":
            temp_df = pd.DataFrame(song_part_main_data[k][0])
            print(temp_df.head())
            break
        else:
            # Print unique values of each column
            song_part_main_data_df = pd.DataFrame(song_part_main_data[k])
            for col in song_part_main_data_df.columns:
                # If not a list
                if type(song_part_main_data_df[col][0]) != list:
                    print(f"{col}: {song_part_main_data_df[col].unique()}")
                else:
                    print(f"{col}: {song_part_main_data_df[col].apply(lambda x: tuple(x)).unique()}")
    print()


-----> CHORDS:
root: [6 1 3 4 7 5]
beat: [ 1.   3.   5.   7.   9.  15.  17.  20.5 22.5 24.5 28.5 30.5 33.  34.5
 36.5 38.5 41.  46.5 49.  52.5 54.5 57.  60.5 62.5]
duration: [2.  6.  3.5 4.  2.5 1.5 5.5]
type: [5 7]
inversion: [0 3]
applied: [0]
adds: [(9,) ()]
omits: [()]
alterations: [()]
suspensions: [()]
pedal: [None]
alternate: ['']
borrowed: [None 'major']
isRest: [False]
recordingEndBeat: [None]

-----> NOTES:
sd: ['1' '7' '5' '6' '4' 'b4' '3']
octave: [ 0 -1]
beat: [ 1.   3.   3.5  4.   5.   6.   6.5  7.   8.   9.  11.  11.5 12.  13.
 14.  14.5 15.  16.  17.  19.  19.5 20.  21.  22.  22.5 23.  24.  24.5
 25.5 26.5 27.5 28.5 33.  35.  35.5 36.  37.  38.  38.5 39.  40.  41.
 43.  43.5 44.  45.  46.  46.5 47.  48.  49.  51.  51.5 52.  53.  54.
 54.5 55.  56.  56.5 57.5 58.5 59.5 60.5]
duration: [2.  0.5 1.  2.5]
isRest: [ True False]
recordingEndBeat: [None]

-----> INACTIVENOTES:
  sd  octave  beat  duration  isRest recordingEndBeat
0  1       0   1.0       2.0    True          

In [7]:
a = [2, 1, 2, 3, 4]

print(sum(a[:0]))

0


In [8]:
print(mu.scale_degree_to_midi_note_number(
        "#3", 
        key_signature=KeySignatureDTO(
            tonic_midi_note_number=mc.inversed_based_midi_note_numbers["A"],
            scale_name="major"
        ), 
        octave=0
    )
)
print(htu.hooktheory_start_beat_to_tick_position(1.5))

74
480


In [9]:
should_run_listing_unique_values_code = False
should_run_searching_words_code = False

In [10]:
song_parts_from_ids_path = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling/DataPreprocessing/song_parts_from_ids"

json_files = []

# List all json file in song_parts_from_ids_path (recursively)
for root, dirs, files in os.walk(song_parts_from_ids_path):
    if not should_run_listing_unique_values_code and not should_run_searching_words_code:
        break

    for file in files:
        if file.endswith(".json"):
            json_files.append(os.path.join(root, file))

print(len(json_files))

0


In [11]:
word_to_search = """super:"""

for json_file in json_files:
    with open(json_file, "r") as f:
        json_data = json.load(f)


    if word_to_search.lower() in str(json_data).lower():
        print(f"{json_data['link']}: {json_file}")

if should_run_listing_unique_values_code:
    pass
else:
    json_files = []

In [12]:
unique_values = {
    "root": set(),
    "type": set(),
    "inversion": set(),
    "applied": set(),
    "adds": set(),
    "omits": set(),
    "alterations": set(),
    "suspensions": set(),
    "pedal": set(),
    "alternate": set(),
    "borrowed": set()
}

unique_keys = {
    "tonic": set(),
    "scale": set(),
}

unique_meter_attrs = {
    "numBeats": set(),
    "beatUnit": set()
}

ignored_keys = set()
for json_file in json_files:
    with open(json_file, "r") as f:
        song_data = json.load(f)

    for song_part in song_data["song_parts"]:
        if song_part["main_data_type"] in ["json"]:
            for chord in song_part["main_data"]["chords"]:
                for k, v in chord.items():
                    # if k == "borrowed":
                    #     if v is not None and len(v) > 0:
                    #         print(chord, song_part["metadata"]["section"], song_data["link"])
                    #     continue
                    # else:
                    #     continue

                    if k in unique_values.keys():
                        if type(v) == list:
                            for v_ in v:
                                if v_ not in unique_values[k]:
                                    print(f"New value: {v} for key: {k} - section: {song_part['metadata']['section']} - song link: {song_data['link']} {json_file}")
                                unique_values[k].add(v_)
                        else:
                            if v not in unique_values[k]:
                                print(f"New value: {v} for key: {k} - section: {song_part['metadata']['section']} - song link: {song_data['link']} {json_file}")
                            unique_values[k].add(v)
                    else:
                        ignored_keys.add(k)

            for key in song_part["main_data"]["keys"]:
                for k, v in key.items():
                    if k in unique_keys.keys():
                        if v not in unique_keys[k]:
                            print(f"New value: {v} for key: {k} - section: {song_part['metadata']['section']} - song link: {song_data['link']} {json_file}")
                        unique_keys[k].add(v)
                    else:
                        ignored_keys.add(k)

            for meter in song_part["main_data"]["meters"]:
                for k, v in meter.items():
                    if k in unique_meter_attrs.keys():
                        if v not in unique_meter_attrs[k]:
                            print(f"New value: {v} ({meter}) for key: {k} - section: {song_part['metadata']['section']} - song link: {song_data['link']} {json_file}")
                        unique_meter_attrs[k].add(v)
                    else:
                        ignored_keys.add(k)

In [13]:
for k, v in unique_values.items():
    print(f"{k}: {v}")

for k, v in unique_keys.items():
    print(f"{k}: {v}")

for k, v in unique_meter_attrs.items():
    print(f"{k}: {v}")    

print("ignored_keys: ", ignored_keys)

root: set()
type: set()
inversion: set()
applied: set()
adds: set()
omits: set()
alterations: set()
suspensions: set()
pedal: set()
alternate: set()
borrowed: set()
tonic: set()
scale: set()
numBeats: set()
beatUnit: set()
ignored_keys:  set()


In [14]:
unique_values_xml = {
    "sd": set(),
    "fb": set(),
    "sec": set(),
    "sus": set(),
    "pedal": set(),
    "alternate": set(),
    "borrowed": set(),
    "chord_duration": set(),
    "start_measure": set(),
    "start_beat": set(),
    "start_beat_abs": set(),
    "isRest": set()
}

unique_keys_xml = set()

unique_scale_xml = set()

unique_meter_attrs_xml = set()

ignored_keys_xml = set()

for json_file in json_files:
    with open(json_file, "r") as f:
        song_data = json.load(f)

    for song_part in song_data["song_parts"]:
        if song_part["main_data_type"] in ["xml"]:
            if "super" in song_part["main_data"]:
                continue
            if type(song_part["main_data"]["theorytab"]["data"]["segment"]) != list:
                song_part["main_data"]["theorytab"]["data"]["segment"] = [song_part["main_data"]["theorytab"]["data"]["segment"]]
            for segment in song_part["main_data"]["theorytab"]["data"]["segment"]:
                if segment["harmony"] == None:
                    continue
                
                chords = segment["harmony"]["chord"]
                
                if type(chords) != list:
                    chords = [chords]

                for chord in chords:
                    for k, v in chord.items():
                        if k in unique_values_xml.keys():
                            if type(v) == list:
                                for v_ in v:
                                    if v_ not in unique_values_xml[k]:
                                        print(f"New value: {v} for key: {k} - section: {song_part['metadata']['section']} - song link: {song_data['link']} {json_file}")
                                    unique_values_xml[k].add(v_)
                            else:
                                if v not in unique_values_xml[k]:
                                    print(f"New value: {v} for key: {k} - section: {song_part['metadata']['section']} - song link: {song_data['link']} {json_file}")
                                unique_values_xml[k].add(v)
                        else:
                            ignored_keys_xml.add(k)

                song_key = song_part["main_data"]["theorytab"]["meta"]["key"]
                if song_key not in unique_keys_xml:
                    print(f"New value: {song_key} for key: key - section: {song_part['metadata']['section']} - song link: {song_data['link']} {json_file}")
                    unique_keys_xml.add(song_key)

                song_mode = song_part["main_data"]["theorytab"]["meta"]["mode"]
                if song_mode not in unique_scale_xml:
                    print(f"New value: {song_mode} for key: mode - section: {song_part['metadata']['section']} - song link: {song_data['link']} {json_file}")
                    unique_scale_xml.add(song_mode)

                meter = song_part["main_data"]["theorytab"]["meta"]["beats_in_measure"]
                if meter not in unique_meter_attrs_xml:
                    print(f"New value: {meter} ({meter}) for key: beats_in_measure - section: {song_part['metadata']['section']} - song link: {song_data['link']} {json_file}")
                    unique_meter_attrs_xml.add(meter)

In [15]:
for k, v in unique_values_xml.items():
    print(f"{k}: {v}")

print(f"key: {unique_keys_xml}")

print(f"mode: {unique_scale_xml}")

print(f"meter: {unique_meter_attrs_xml}")

print("ignored_keys: ", ignored_keys_xml)

sd: set()
fb: set()
sec: set()
sus: set()
pedal: set()
alternate: set()
borrowed: set()
chord_duration: set()
start_measure: set()
start_beat: set()
start_beat_abs: set()
isRest: set()
key: set()
mode: set()
meter: set()
ignored_keys:  set()


In [16]:
hooktheory_json_3["song_parts"][0]["main_data"]["chords"] = [
    hooktheory_json_3["song_parts"][1]["main_data"]["chords"][2]
]

for_you_dto = hooktheory_json_song_part_to_midi_dto_converter(
    hooktheory_json_3["song_parts"][0]
)

for_you_output_path = f"{midi_source_path}/Ideas/hooktheory/output/for-you.mid"
midi_dto_to_midi_file_converter(for_you_dto, for_you_output_path)

{'song_part_id': 'JkmZkWZjgqn', 'metadata': {'ID': 882246, 'user_id': 103696, 'dateModified': '2021-08-16 19:11:54+00:00', 'HTID': None, 'artist': 'Robert Glasper', 'song': 'For You', 'section': 'Verse', 'revision': 0, 'chords': None, 'youTubeID': 'dvo_8e5ukQk', 'dotDNAbin': None, 'xmlType': None, 'SIF': None, 'SInE': None, 'SInD': None, 'SInED': None, 'SInA': None, 'beatsInMeasure': None, 'songKey': None, 'isPrivate': '0', 'views': 0, 'hasMelody': None, 'hasChords': None, 'isAnalysis': 0, 'voteup': 0, 'votedown': 0, 'score': 0, 'songURL': 'for-you', 'artistURL': 'robert-glasper', 'sectionURL': 'verse', 'editComments': '', 'visible': 1, 'wikiorder': 1, 'bIsPaired': 0, 'YouTubeVideoID': None, 'bpm': None, 'mode': 1, 'borrowedFixed': '0', 'SInEDFixed': '0', 'user_id_editor': 272859}, 'main_data_type': 'json', 'main_data': {'version': 1, 'chords': [{'root': 5, 'beat': 8, 'duration': 7, 'type': 13, 'inversion': 0, 'applied': 0, 'adds': [], 'omits': [], 'alterations': ['#11'], 'suspensions'

In [17]:
chord_example = hooktheory_json_3["song_parts"][0]["main_data"]["chords"][0]

print(chord_example)

{'root': 5, 'beat': 8, 'duration': 7, 'type': 13, 'inversion': 0, 'applied': 0, 'adds': [], 'omits': [], 'alterations': ['#11'], 'suspensions': [], 'pedal': None, 'alternate': '', 'borrowed': 'lydian', 'isRest': False, 'recordingEndBeat': None}


In [18]:
alterations = ["b5", "13", "#15", "#b7"]
print(alterations)
for alt_note in alterations:
    letter_idx = 0
    n_sharp = 0
    n_flat = 0

    for letter_idx in range(0, len(alt_note)):
        if alt_note[letter_idx] == "b":
            n_flat += 1
        elif alt_note[letter_idx] == "#":
            n_sharp += 1
        else:
            break
    
    scale_degree = alt_note[letter_idx:]

    print(f"letter_idx: {letter_idx}, n_sharp: {n_sharp}, n_flat: {n_flat}, scale_degree: {scale_degree}")

['b5', '13', '#15', '#b7']
letter_idx: 1, n_sharp: 0, n_flat: 1, scale_degree: 5
letter_idx: 0, n_sharp: 0, n_flat: 0, scale_degree: 13
letter_idx: 1, n_sharp: 1, n_flat: 0, scale_degree: 15
letter_idx: 2, n_sharp: 1, n_flat: 1, scale_degree: 7


In [19]:
zedd_dto = hooktheory_json_song_part_to_midi_dto_converter(
    hooktheory_json_1["song_parts"][0]
)

zedd_output_path = f"{midi_source_path}/Ideas/hooktheory/output/zedd.mid"
midi_dto_to_midi_file_converter(zedd_dto, zedd_output_path)

MIDI file saved successfully: /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/hooktheory/output/zedd.mid


In [20]:
song_name = "for-you"
hooktheory_json_file_path = f"{midi_source_path}/Ideas/hooktheory/data/{song_name}.json"

with open(hooktheory_json_file_path, "r") as f:
    hooktheory_json = json.load(f)

part_idx = 0

dua_dto = hooktheory_json_song_part_to_midi_dto_converter(
    hooktheory_json["song_parts"][part_idx]
)

dua_output_path = f"{midi_source_path}/Ideas/hooktheory/output/{song_name}_{part_idx}.mid"
midi_dto_to_midi_file_converter(dua_dto, dua_output_path)

Ignored applied value: 5
MIDI file saved successfully: /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/hooktheory/output/super-smash-bros-melee---yoshis-story_2.mid


In [21]:
taylor_dto = hooktheory_json_song_part_to_midi_dto_converter(
    hooktheory_json_2["song_parts"][0]
)

taylor_output_path = f"{midi_source_path}/Ideas/hooktheory/output/taylor.mid"
midi_dto_to_midi_file_converter(taylor_dto, taylor_output_path)

MIDI file saved successfully: /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/hooktheory/output/taylor.mid


In [22]:
for i in range(10, 10):
    print(i)

a = {
    1 : "a",
    2 : "b",
}

a[89897] = "c"

print(a)

{1: 'a', 2: 'b', 89897: 'c'}


In [23]:
borrowed = [0, 2, 4, 5, 8, 9, 11]

n_notes = len(borrowed)

scale_formula = [
    (
        borrowed[(i + 1) % n_notes] + mc.n_semitones_per_octave - borrowed[i]
    ) % mc.n_semitones_per_octave
    for i in range(0, n_notes)
]

print(scale_formula)

[2, 2, 1, 3, 1, 2, 1]
